In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.metrics import confusion_matrix,classification_report
import numpy as np
from sklearn.metrics import accuracy_score, f1_score
from imblearn.over_sampling import SMOTE
from sklearn.impute import SimpleImputer
import time

import functions


ALL = False
NO_SMOTE = True
SUB_PROCESSED_DIR = 'processed'
TEST_RATIO = 0.1

dtypes = {
    'msisdn': 'str',
    'start_time': 'str',
    'end_time': 'str',
    'call_event': 'category',
    'other_party': 'str',
    'ismultimedia': 'category',
    'home_area_code': 'str',
    'visit_area_code': 'str',
    'called_home_code': 'str',
    'called_code': 'str',
    'a_serv_type': 'int',
    'long_type1': 'int',
    'roam_type': 'int',
    'a_product_id': 'str',
    'open_datetime': 'str',
    'call_duration': 'int32',
    'cfee': 'float64',
    'lfee': 'float64',
    'hour': 'int8',
    'dayofweek': 'int',
    'phone1_type': 'int',
    'phone2_type': 'int',
    'phone1_loc_city': 'str',
    'phone1_loc_province': 'str',
    'phone2_loc_city': 'str',
    'phone2_loc_province': 'str',
    'update_time': 'str',
    'date': 'str',
    'date_c': 'str',

    "phone1_loc_city_lat": "float64",
    "phone1_loc_city_lon": "float64",
    "phone2_loc_city_lat": "float64",
    "phone2_loc_city_lon": "float64",
}

# 判断 processed 文件夹是否存在
import os
if not os.path.exists(f'../self_data/{SUB_PROCESSED_DIR}'):
    print("Creating processed data folder...")
    # 读取CSV文件
    labeled_data = pd.read_csv('../self_data/all_trainSet_res.csv', dtype=dtypes)
    labels = pd.read_csv('../self_data/all_trainSet_ans.csv', dtype=dtypes)
    validation_data = pd.read_csv('../self_data/sorted_validationSet_res_with_head.csv', dtype=dtypes)

    # 按照 msisdn 切分 train_data 和 test_data
    train_data_msisdn, test_data_msisdn = train_test_split(labels['msisdn'], test_size=TEST_RATIO, random_state=42, stratify=labels['is_sa'])
    train_data = labeled_data[labeled_data['msisdn'].isin(train_data_msisdn)]
    train_labels = labels[labels['msisdn'].isin(train_data_msisdn)]
    assert len(train_data['msisdn'].unique()) == len(train_data_msisdn)

    test_data = labeled_data[labeled_data['msisdn'].isin(test_data_msisdn)]
    test_labels = labels[labels['msisdn'].isin(test_data_msisdn)]
    assert len(test_data['msisdn'].unique()) == len(test_data_msisdn)

    train_data, train_labels, labels_aug = functions.augment_data_parallel(train_data, train_labels, test_labels)

    # save
    print("Saving processed data...")
    os.makedirs(f'../self_data/{SUB_PROCESSED_DIR}', exist_ok=True)
    train_data.to_pickle(f'../self_data/{SUB_PROCESSED_DIR}/train_data.pkl')
    train_labels.to_pickle(f'../self_data/{SUB_PROCESSED_DIR}/train_labels.pkl')
    test_data.to_pickle(f'../self_data/{SUB_PROCESSED_DIR}/test_data.pkl')
    test_labels.to_pickle(f'../self_data/{SUB_PROCESSED_DIR}/test_labels.pkl')
    labels_aug.to_pickle(f'../self_data/{SUB_PROCESSED_DIR}/labels_aug.pkl')
    validation_data.to_pickle(f'../self_data/{SUB_PROCESSED_DIR}/validation_data.pkl')

else:
    print("Reading processed data...")
    train_data = pd.read_pickle(f'../self_data/{SUB_PROCESSED_DIR}/train_data.pkl')
    train_labels = pd.read_pickle(f'../self_data/{SUB_PROCESSED_DIR}/train_labels.pkl')
    test_data = pd.read_pickle(f'../self_data/{SUB_PROCESSED_DIR}/test_data.pkl')
    test_labels = pd.read_pickle(f'../self_data/{SUB_PROCESSED_DIR}/test_labels.pkl')
    labels_aug = pd.read_pickle(f'../self_data/{SUB_PROCESSED_DIR}/labels_aug.pkl')
    validation_data = pd.read_pickle(f'../self_data/{SUB_PROCESSED_DIR}/validation_data.pkl')

labeled_data_aug = pd.concat([train_data, test_data], ignore_index=True).reindex()
assert len(labeled_data_aug['msisdn'].unique()) == len(labels_aug['msisdn'].unique())

# 转换时间格式
labeled_data_aug['start_time'] = pd.to_datetime(labeled_data_aug['start_time'], format='%Y%m%d%H%M%S')
labeled_data_aug['end_time'] = pd.to_datetime(labeled_data_aug['end_time'], format='%Y%m%d%H%M%S')
labeled_data_aug['open_datetime'] = pd.to_datetime(labeled_data_aug['open_datetime'], format='%Y%m%d%H%M%S')
labeled_data_aug['update_time'] = pd.to_datetime(labeled_data_aug['update_time'])
labeled_data_aug['date'] = pd.to_datetime(labeled_data_aug['date'])

validation_data['start_time'] = pd.to_datetime(validation_data['start_time'], format='%Y%m%d%H%M%S')
validation_data['end_time'] = pd.to_datetime(validation_data['end_time'], format='%Y%m%d%H%M%S')
validation_data['open_datetime'] = pd.to_datetime(validation_data['open_datetime'], format='%Y%m%d%H%M%S',errors='coerce')
validation_data['update_time'] = pd.to_datetime(validation_data['update_time'])
validation_data['date'] = pd.to_datetime(validation_data['date'])

In [ ]:
# 为每条记录添加start_time_diff，记录 start_time 与上一条记录的 start_time 之差 (单位：秒)
start_time_diff = labeled_data_aug.groupby('msisdn')['start_time'].diff().dt.total_seconds().fillna(0).reset_index(drop=True)
# 将该列加入到数据集中
labeled_data_aug['start_time_diff'] = start_time_diff.copy()
start_time_diff = validation_data.groupby('msisdn')['start_time'].diff().dt.total_seconds().fillna(0).reset_index(drop=True)
validation_data['start_time_diff'] = start_time_diff.copy()

数据特征处理

In [ ]:
labeled_aug_features = functions.aggregate_features_parallel(labeled_data_aug)
validation_features = functions.aggregate_features_parallel(validation_data)

# 合并标签数据
labeled_aug_features = labeled_aug_features.merge(labels_aug, on='msisdn', how='left')

In [ ]:
functions.get_nan(labeled_aug_features), functions.get_nan(validation_features)

In [ ]:
# 一般只有 std 会出现 nan 值故所有的 nan 值填充为 0
labeled_aug_features = labeled_aug_features.fillna(0)
validation_features = validation_features.fillna(0)
functions.get_nan(labeled_aug_features), functions.get_nan(validation_features)

In [ ]:
X = labeled_aug_features.drop(['msisdn'], axis=1)
y = labeled_aug_features['is_sa']
X_validation = validation_features.drop(['msisdn'], axis=1)

n_sample = y.shape[0]
n_pos_sample = y[y ==1].shape[0]
n_neg_sample = y[y == 0].shape[0]
print('样本个数：{}; 正样本占{:.2%}; 负样本占{:.2%}'.format(n_sample,
                                                   n_pos_sample / n_sample,
                                                   n_neg_sample / n_sample))
print('特征维数：', X.shape[1])

In [ ]:
labeled_aug_features.columns

In [ ]:
# TODO use all_X to impute
imputer = SimpleImputer(strategy='most_frequent')
X = imputer.fit_transform(X)

In [ ]:
imputer2 = SimpleImputer(strategy='most_frequent')
X_validation = imputer2.fit_transform(X_validation)

In [ ]:
assert len(train_labels) + len(test_labels) == len(labeled_aug_features)
len(train_labels), len(test_labels)

In [ ]:
from sklearn.utils import shuffle
# 将 msisdn 和 is_sa 并入 X 再划分
train_data_msisdn = train_labels['msisdn']
test_data_msisdn = test_labels['msisdn']
X_df = pd.DataFrame(X, columns=labeled_aug_features.drop(['msisdn'], axis=1).columns)
X_df = pd.concat([labeled_aug_features[['msisdn']], X_df], axis=1)
train_set = X_df[X_df['msisdn'].isin(train_data_msisdn)][X_df.columns[1:]]
test_set = X_df[X_df['msisdn'].isin(test_data_msisdn)][X_df.columns[1:]]

print(f"1 samples / 0 samples in train set: {len(train_set[train_set['is_sa'] == 1])} / {len(train_set[train_set['is_sa'] == 0])}")
print(f"1 samples / 0 samples in test set: {len(test_set[test_set['is_sa'] == 1])} / {len(test_set[test_set['is_sa'] == 0])}")

if ALL:
    train_len = len(test_set) + len(train_set)
    test_len = 0
else:
    train_len, test_len = len(train_set), len(test_set)

In [ ]:
columns = labeled_aug_features.columns.tolist()
columns.remove('msisdn')
valid_set = np.c_[X_validation, np.zeros(X_validation.shape[0])]
valid_set = pd.DataFrame(valid_set, columns=columns)
valid_set['is_sa'] = -1

In [ ]:
train_set.head()

In [ ]:
test_set.describe()

In [ ]:
all_set = pd.concat([train_set, test_set, valid_set], axis=0).reset_index(drop=True)
labeled_data_len = train_set.shape[0] + test_set.shape[0]

In [ ]:
test_set.shape, train_set.shape, valid_set.shape, all_set.shape

In [ ]:
labeled_set, valid_set = all_set.iloc[:labeled_data_len].copy(), all_set.iloc[labeled_data_len:].copy()
labeled_set.reset_index(drop=True, inplace=True)
valid_set.reset_index(drop=True, inplace=True)
# 有一些值在SMOTE后对数变换后为 NaN，需要删除这些数据
print(labeled_set.isnull().sum().sum())
labeled_set = labeled_set.dropna()
print(labeled_set.isnull().sum().sum())
assert valid_set.shape[0] == validation_features.shape[0]

# 重新划分训练集和测试集
if not ALL:
    train_set, test_set = labeled_set.iloc[:train_len].copy(), labeled_set.iloc[train_len:].copy()
    train_set.reset_index(drop=True, inplace=True)
    test_set.reset_index(drop=True, inplace=True)


    remove_columns = ['cfee+std', 'start_time_diff+start_time_diff_std', 'lfee+mean', 'lfee+sum', 'lfee+std', 'start_time_diff+max']
    # remove_columns = ['cfee+std', 'lfee+mean', 'lfee+sum', 'lfee+std']
    train_set = train_set.drop(remove_columns, axis=1)
    test_set = test_set.drop(remove_columns, axis=1)
    valid_set = valid_set.drop(remove_columns, axis=1)
    assert train_set.shape[1] == test_set.shape[1] == valid_set.shape[1]
   
else:
    remove_columns = ['cfee+std', 'start_time_diff+start_time_diff_std', 'lfee+mean', 'lfee+sum', 'lfee+std']
    labeled_set = labeled_set.drop(remove_columns, axis=1)
    valid_set = valid_set.drop(remove_columns, axis=1)
    test_set = test_set.drop(remove_columns, axis=1)
    assert labeled_set.shape[1] == valid_set.shape[1] == test_set.shape[1]
    
# 对采样数据做 smote
if not NO_SMOTE:
    smote = SMOTE(random_state=42)    # 处理过采样的方法
    X_train, y_train = smote.fit_resample(train_set.drop(['is_sa'], axis=1), train_set['is_sa'])
    train_set = pd.concat([X_train, y_train], axis=1)
    print('通过SMOTE方法平衡正负样本后')
    n_sample = y_train.shape[0]
    n_pos_sample = y_train[y_train == 1].shape[0]
    n_neg_sample = y_train[y_train == 0].shape[0]
    print('样本个数：{}; 正样本占{:.2%}; 负样本占{:.2%}'.format(n_sample,
                                                    n_pos_sample / n_sample,
                                                    n_neg_sample / n_sample))

In [ ]:
train_set.shape, test_set.shape, valid_set.shape

In [ ]:
from autogluon.tabular import TabularPredictor

if not ALL:
    model = TabularPredictor(label='is_sa', eval_metric='f1', problem_type='binary').fit(train_set, presets='medium_quality', time_limit=3600)
else:
    model = TabularPredictor(label='is_sa', eval_metric='f1', problem_type='binary').fit(labeled_set, presets='best_quality', num_bag_folds=10, time_limit=3600)

In [ ]:
if not ALL:
    print(model.evaluate(test_set))

In [ ]:
feature_importance = model.feature_importance(test_set if not ALL else labeled_set)
print(feature_importance)
feature_importance

In [ ]:
# leaderboard
if not ALL:
    leaderboard = model.leaderboard(test_set, silent=True)
    print(leaderboard)
else:
    leaderboard = model.leaderboard(labeled_set, silent=True)
    print(leaderboard)
leaderboard

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report

if not ALL:
    y_pred = model.predict(test_set)
    y_true = test_set['is_sa']
    
    print(classification_report(y_true, y_pred))
    
    cm = confusion_matrix(y_true, y_pred)
    
    plt.figure(figsize=(10, 7))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Predicted Negative', 'Predicted Positive'], yticklabels=['Actual Negative', 'Actual Positive'])
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title('Confusion Matrix')
    plt.tight_layout()
    os.makedirs("../vis", exist_ok=True)
    plt.savefig("../vis/confusion_matrix.png")
    plt.show()

In [ ]:
# 模型决策阈值微调
threadhold = 0.2
if not ALL:
    y_pred_proba = model.predict_proba(test_set)
    # print(y_pred_proba)
    y_pred = (y_pred_proba.iloc[:, 1] > threadhold).astype(int)
    print(classification_report(y_true, y_pred))
    print(confusion_matrix(y_true, y_pred))

In [ ]:
# 预测
y_validation_pred = model.predict(valid_set.drop('is_sa', axis=1))

# 将预测结果与 msisdn 对应起来
validation_results = validation_features[['msisdn']].copy()
validation_results['is_sa'] = y_validation_pred.astype(int)

print(validation_results.describe())

# 保存结果到CSV文件
time_str = time.strftime("%Y%m%d%H%M%S", time.localtime())
file_name = './valid_large_data_{}.csv'.format(time_str) if ALL else './valid_small_data_{}.csv'.format(time_str)
validation_results.to_csv(file_name, index=False)
print(file_name)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_curve, average_precision_score

# f1 最佳阈值调整，找到 best_threshold
if not ALL:
    y_true = test_set['is_sa']
    y_pred_proba = model.predict_proba(test_set).iloc[:, 1]  # 获取正类的预测概率

    # 计算不同阈值下的精确率和召回率
    precision, recall, thresholds = precision_recall_curve(y_true, y_pred_proba)

    # 计算平均精确率
    average_precision = average_precision_score(y_true, y_pred_proba)



    # 绘制 PR 曲线
    plt.figure()
    plt.plot(recall, precision, label=f'PR curve (area = {average_precision:.2f})')
    # 画出最佳 f1 分数对应的阈值
    best_index = np.argmax(2 * precision * recall / (precision + recall))
    best_threshold = thresholds[best_index]
    f1 = 2 * precision[best_index] * recall[best_index] / (precision[best_index] + recall[best_index])
    plt.plot(recall[best_index], precision[best_index], 'ro', label=f'Best F1 {f1} Threshold: {best_threshold:.2f}')
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('Precision-Recall Curve')
    plt.legend(loc='best')
    plt.grid(True)
    plt.show()

In [ ]:
print("best threshold: ", best_threshold)
# 使用最佳决策阈值进行预测
y_validation_pred_proba = model.predict_proba(valid_set.drop('is_sa', axis=1))
y_validation_pred = (y_validation_pred_proba.iloc[:, 1] >= best_threshold).astype(int)

# 将预测结果与 msisdn 对应起来
validation_results = validation_features[['msisdn']].copy()
validation_results['is_sa'] = y_validation_pred.astype(int)

print(validation_results.describe())

# 保存结果到CSV文件
import time
time_str = time.strftime("%Y%m%d%H%M%S", time.localtime())
file_name = './valid_large_data_{}.csv'.format(time_str) if ALL else './valid_small_data_{}.csv'.format(time_str)
validation_results.to_csv(file_name, index=False)
print(file_name)